In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler  

In [2]:
df_yield = pd.read_csv('yield.csv')

In [3]:
df_yield.shape

(56717, 12)

In [4]:
df_yield.head(5)

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value
0,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1961,1961,hg/ha,14000
1,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1962,1962,hg/ha,14000
2,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1963,1963,hg/ha,14260
3,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1964,1964,hg/ha,14257
4,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1965,1965,hg/ha,14400


In [5]:
df_yield.info

<bound method DataFrame.info of       Domain Code Domain  Area Code         Area  Element Code Element  \
0              QC  Crops          2  Afghanistan          5419   Yield   
1              QC  Crops          2  Afghanistan          5419   Yield   
2              QC  Crops          2  Afghanistan          5419   Yield   
3              QC  Crops          2  Afghanistan          5419   Yield   
4              QC  Crops          2  Afghanistan          5419   Yield   
...           ...    ...        ...          ...           ...     ...   
56712          QC  Crops        181     Zimbabwe          5419   Yield   
56713          QC  Crops        181     Zimbabwe          5419   Yield   
56714          QC  Crops        181     Zimbabwe          5419   Yield   
56715          QC  Crops        181     Zimbabwe          5419   Yield   
56716          QC  Crops        181     Zimbabwe          5419   Yield   

       Item Code   Item  Year Code  Year   Unit  Value  
0             56  Maiz

In [6]:
df_yield.rename( columns={"Value": "hg/ha_yield"}, inplace=True)

In [7]:
df_yield.head(5)

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,hg/ha_yield
0,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1961,1961,hg/ha,14000
1,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1962,1962,hg/ha,14000
2,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1963,1963,hg/ha,14260
3,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1964,1964,hg/ha,14257
4,QC,Crops,2,Afghanistan,5419,Yield,56,Maize,1965,1965,hg/ha,14400


In [8]:
df_yield = df_yield.drop(['Year Code','Element Code', 'Element','Year Code','Area Code','Domain Code', 'Domain','Unit','Item Code'], axis=1) 

In [9]:
df_yield.head(5)

,Area,Item,Year,hg/ha_yield
0,Afghanistan,Maize,1961,14000
1,Afghanistan,Maize,1962,14000
2,Afghanistan,Maize,1963,14260
3,Afghanistan,Maize,1964,14257
4,Afghanistan,Maize,1965,14400


In [10]:
df_yield = df_yield.dropna()

In [11]:
##Load Climate Data

In [12]:
df_rain = pd.read_csv('rainfall.csv')

In [13]:
df_rain.head(5)

,Area,Year,average_rain_fall_mm_per_year
0,Afghanistan,1985,327
1,Afghanistan,1986,327
2,Afghanistan,1987,327
3,Afghanistan,1989,327
4,Afghanistan,1990,327


In [14]:
df_rain.head()

,Area,Year,average_rain_fall_mm_per_year
0,Afghanistan,1985,327
1,Afghanistan,1986,327
2,Afghanistan,1987,327
3,Afghanistan,1989,327
4,Afghanistan,1990,327


In [15]:
df_rain['average_rain_fall_mm_per_year'] = pd.to_numeric(df_rain['average_rain_fall_mm_per_year'],errors = 'coerce')  

In [16]:
df_rain = df_rain.dropna()

In [17]:
df_main = pd.merge(df_yield, df_rain, on=['Year','Area'], how='outer')

In [18]:
df_main.tail(5)

,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year
58057,West Bank and Gaza,NaN,2016,NaN,402.0
58058,West Bank and Gaza,NaN,2017,NaN,402.0
58059,Yemen,NaN,2017,NaN,167.0
58060,Zambia,NaN,2017,NaN,1020.0
58061,Zimbabwe,NaN,2017,NaN,657.0


In [19]:
df_main.dropna()

,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year
96,Afghanistan,Maize,1985,16652.0,327.0
97,Afghanistan,Potatoes,1985,140909.0,327.0
98,Afghanistan,"Rice, paddy",1985,22482.0,327.0
99,Afghanistan,Wheat,1985,12277.0,327.0
100,Afghanistan,Maize,1986,16875.0,327.0
...,...,...,...,...,...
56712,Zimbabwe,"Rice, paddy",2016,22828.0,657.0
56713,Zimbabwe,Sorghum,2016,2688.0,657.0
56714,Zimbabwe,Soybeans,2016,12727.0,657.0
56715,Zimbabwe,Sweet potatoes,2016,22212.0,657.0


In [20]:
dataframe_pesticide = pd.read_csv('pesticides.csv')

In [21]:
dataframe_pesticide.head()  

,Domain,Area,Element,Item,Year,Unit,Value
0,Pesticides Use,Albania,Use,Pesticides (total),1990,tonnes of active ingredients,121.0
1,Pesticides Use,Albania,Use,Pesticides (total),1991,tonnes of active ingredients,121.0
2,Pesticides Use,Albania,Use,Pesticides (total),1992,tonnes of active ingredients,121.0
3,Pesticides Use,Albania,Use,Pesticides (total),1993,tonnes of active ingredients,121.0
4,Pesticides Use,Albania,Use,Pesticides (total),1994,tonnes of active ingredients,201.0


In [22]:
dataframe_pesticide = dataframe_pesticide.rename(index=str, columns={"Value": "pesticides_tonnes"}) 

In [23]:
dataframe_pesticide = dataframe_pesticide.drop(['Element','Domain','Unit','Item'], axis=1)  

In [24]:
dataframe_pesticide.head() 

,Area,Year,pesticides_tonnes
0,Albania,1990,121.0
1,Albania,1991,121.0
2,Albania,1992,121.0
3,Albania,1993,121.0
4,Albania,1994,201.0


In [25]:
df_main = pd.merge(df_main,dataframe_pesticide,on=['Year','Area'])

In [26]:
df_main.head(5)

,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes
0,Albania,Maize,1990,36613.0,1485.0,121.0
1,Albania,Potatoes,1990,66667.0,1485.0,121.0
2,Albania,"Rice, paddy",1990,23333.0,1485.0,121.0
3,Albania,Sorghum,1990,12500.0,1485.0,121.0
4,Albania,Soybeans,1990,7000.0,1485.0,121.0


In [27]:
dataframe_temp= pd.read_csv('temp.csv')  

In [28]:
dataframe_temp.head(5)  

,year,country,avg_temp
0,1849,Côte D'Ivoire,25.58
1,1850,Côte D'Ivoire,25.52
2,1851,Côte D'Ivoire,25.67
3,1852,Côte D'Ivoire,NaN
4,1853,Côte D'Ivoire,NaN


In [29]:
dataframe_temp =dataframe_temp.dropna()

In [30]:
dataframe_temp = dataframe_temp.rename(index=str, columns={"year": "Year", "country": 'Area'})

In [31]:
dataframe_temp.head()

,Year,Area,avg_temp
0,1849,Côte D'Ivoire,25.58
1,1850,Côte D'Ivoire,25.52
2,1851,Côte D'Ivoire,25.67
7,1856,Côte D'Ivoire,26.28
8,1857,Côte D'Ivoire,25.17


In [32]:
df_main = pd.merge(df_main,dataframe_temp,on=['Year','Area'])

In [33]:
df_main.head(5)

,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp
0,Albania,Maize,1990,36613.0,1485.0,121.0,16.37
1,Albania,Potatoes,1990,66667.0,1485.0,121.0,16.37
2,Albania,"Rice, paddy",1990,23333.0,1485.0,121.0,16.37
3,Albania,Sorghum,1990,12500.0,1485.0,121.0,16.37
4,Albania,Soybeans,1990,7000.0,1485.0,121.0,16.37


In [34]:
df_main.isnull().sum()

Area                                0
Item                                0
Year                                0
hg/ha_yield                         0
average_rain_fall_mm_per_year    1507
pesticides_tonnes                   0
avg_temp                            0
dtype: int64

In [35]:
df_main.dropna()

,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp
0,Albania,Maize,1990,36613.0,1485.0,121.00,16.37
1,Albania,Potatoes,1990,66667.0,1485.0,121.00,16.37
2,Albania,"Rice, paddy",1990,23333.0,1485.0,121.00,16.37
3,Albania,Sorghum,1990,12500.0,1485.0,121.00,16.37
4,Albania,Soybeans,1990,7000.0,1485.0,121.00,16.37
...,...,...,...,...,...,...,...
29744,Zimbabwe,"Rice, paddy",2013,22581.0,657.0,2550.07,19.76
29745,Zimbabwe,Sorghum,2013,3066.0,657.0,2550.07,19.76
29746,Zimbabwe,Soybeans,2013,13142.0,657.0,2550.07,19.76
29747,Zimbabwe,Sweet potatoes,2013,22222.0,657.0,2550.07,19.76


In [36]:
df_main.isnull().sum()

Area                                0
Item                                0
Year                                0
hg/ha_yield                         0
average_rain_fall_mm_per_year    1507
pesticides_tonnes                   0
avg_temp                            0
dtype: int64

In [37]:
df_main=df_main.dropna()  

In [38]:
df_main.isnull().sum()

Area                             0
Item                             0
Year                             0
hg/ha_yield                      0
average_rain_fall_mm_per_year    0
pesticides_tonnes                0
avg_temp                         0
dtype: int64

In [39]:
df_main.groupby('Item').count()  

,Area,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp
Item,,,,,,
Cassava,2045,2045,2045,2045,2045,2045
Maize,4121,4121,4121,4121,4121,4121
Plantains and others,556,556,556,556,556,556
Potatoes,4276,4276,4276,4276,4276,4276
"Rice, paddy",3388,3388,3388,3388,3388,3388
Sorghum,3039,3039,3039,3039,3039,3039
Soybeans,3223,3223,3223,3223,3223,3223
Sweet potatoes,2890,2890,2890,2890,2890,2890
Wheat,3857,3857,3857,3857,3857,3857


In [40]:
df_main_onehot = pd.get_dummies(df_main, columns=['Area',"Item"], prefix = ['Country',"Item"])

In [41]:
features=df_main_onehot.loc[:, df_main_onehot.columns != 'hg/ha_yield']  

In [42]:
label=df_main['hg/ha_yield']  
features.head() 

,Year,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp,Country_Albania,Country_Algeria,Country_Angola,Country_Argentina,Country_Armenia,Country_Australia,...,Item_Cassava,Item_Maize,Item_Plantains and others,Item_Potatoes,"Item_Rice, paddy",Item_Sorghum,Item_Soybeans,Item_Sweet potatoes,Item_Wheat,Item_Yams
0,1990,1485.0,121.0,16.37,True,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,1990,1485.0,121.0,16.37,True,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,1990,1485.0,121.0,16.37,True,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,1990,1485.0,121.0,16.37,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,1990,1485.0,121.0,16.37,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [43]:
features = features.drop(['Year'], axis=1)  

In [44]:
features.head(3)

,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp,Country_Albania,Country_Algeria,Country_Angola,Country_Argentina,Country_Armenia,Country_Australia,Country_Austria,...,Item_Cassava,Item_Maize,Item_Plantains and others,Item_Potatoes,"Item_Rice, paddy",Item_Sorghum,Item_Soybeans,Item_Sweet potatoes,Item_Wheat,Item_Yams
0,1485.0,121.0,16.37,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,1485.0,121.0,16.37,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,1485.0,121.0,16.37,True,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


In [45]:
scaler = MinMaxScaler()  

In [46]:
features = scaler.fit_transform(features)

In [47]:
from sklearn.model_selection import train_test_split  
train_data, test_data, train_labels, test_labels = train_test_split(features, label, test_size=0.2, random_state=42)  

In [48]:
from sklearn.metrics import r2_score  
def compare_models(model):  
    model_name = model.__class__.__name__  
    fit=model.fit(train_data,train_labels)  
    y_pred=fit.predict(test_data)  
    r2=r2_score(test_labels,y_pred)  
    return([model_name,r2]) 

In [49]:
from sklearn.ensemble import RandomForestRegressor  
from sklearn.ensemble import GradientBoostingRegressor  
from sklearn import svm  
from sklearn.tree import DecisionTreeRegressor

In [50]:
models = [  
    GradientBoostingRegressor(n_estimators=200, max_depth=3, random_state=0), 
    RandomForestRegressor(n_estimators=200, max_depth=3, random_state=0),  
    svm.SVR(),  
   DecisionTreeRegressor()  
]

In [51]:
model_train=list(map(compare_models,models))

In [52]:
print(*model_train, sep = "\n")

['GradientBoostingRegressor', 0.8959545600619471]
['RandomForestRegressor', 0.6807690552605921]
['SVR', -0.19466686625412555]
['DecisionTreeRegressor', 0.9605387327397786]


In [53]:
df_main_onehot.head()

,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp,Country_Albania,Country_Algeria,Country_Angola,Country_Argentina,Country_Armenia,...,Item_Cassava,Item_Maize,Item_Plantains and others,Item_Potatoes,"Item_Rice, paddy",Item_Sorghum,Item_Soybeans,Item_Sweet potatoes,Item_Wheat,Item_Yams
0,1990,36613.0,1485.0,121.0,16.37,True,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,1990,66667.0,1485.0,121.0,16.37,True,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,1990,23333.0,1485.0,121.0,16.37,True,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,1990,12500.0,1485.0,121.0,16.37,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,1990,7000.0,1485.0,121.0,16.37,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [54]:
# converting test data to columns from the dataframe and omitting the values for "hg/ha yield," which the machine learning model should be predicting  
dataframe_test=pd.DataFrame(test_data,columns=df_main_onehot.loc[:, df_main_onehot.columns != 'hg/ha_yield'].columns)  
  
# utilizing the stack function to pivot the columns of the current dataframe and return a reshaped dataframe  
  
cntry=dataframe_test[[col for col in dataframe_test.columns if 'Country' in col]].stack()[dataframe_test[[col for col in dataframe_test.columns if 'Country' in col]].stack()>0]  
cntrylist=list(pd.DataFrame(cntry).index.get_level_values(1))  
countries=[i.split("_")[1] for i in cntrylist]  
itm=dataframe_test[[col for col in dataframe_test.columns if 'Item' in col]].stack()[dataframe_test[[col for col in dataframe_test.columns if 'Item' in col]].stack()>0]  
itmlist=list(pd.DataFrame(itm).index.get_level_values(1))  
items=[i.split("_")[1] for i in itmlist]  
  
  
dataframe_test.head()

ValueError: Shape of passed values is (5649, 114), indices imply (5649, 115)

In [ ]:
dataframe_test.drop([col for col in dataframe_test.columns if 'Item' in col],axis=1,inplace=True)  
dataframe_test.drop([col for col in dataframe_test.columns if 'Country' in col],axis=1,inplace=True)  
dataframe_test.head() 

In [ ]:
dataframe_test['Country']=countries  
dataframe_test['Item']=items  
dataframe_test.head() 

In [ ]:
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
clf=DecisionTreeRegressor()  
model=clf.fit(train_data,train_labels)  
  
dataframe_test["yield_predicted"]= model.predict(test_data)  
dataframe_test["yield_actual"]=pd.DataFrame(test_labels)["hg/ha_yield"].tolist()  
test_group=dataframe_test.groupby("Item")  
  
# So let's compare the model's actual values to its predictions.  
  
fig, ax = plt.subplots()  
  
ax.scatter(dataframe_test["yield_actual"], dataframe_test["yield_predicted"],edgecolors=(0, 0, 0))
  
ax.set_xlabel('Actual')  
ax.set_ylabel('Predicted')  
ax.set_title("Actual vs Predicted")  
plt.show()  

In [ ]:
import seaborn as sns
varimp= {'imp':model.feature_importances_,'names':df_main_onehot.columns[df_main_onehot.columns!="hg/ha_yield"]}  
  
a4_dims = (8.27,16.7)  
fig, ax = plt.subplots(figsize=a4_dims)  
df=pd.DataFrame.from_dict(varimp)  
df.sort_values(ascending=False,by=["imp"],inplace=True)  
dfdf=df.dropna()  
sns.barplot(x="imp",y="names",hue='names', palette="vlag",data=df,orient="h",ax=ax, legend=False);  

In [ ]:
#7 most important factors that affect crops  
a4_dims = (16.7, 8.27)  
  
fig, ax = plt.subplots(figsize=a4_dims)  
df=pd.DataFrame.from_dict(varimp)  
df.sort_values(ascending=False,by=["imp"],inplace=True)  
dfdf=df.dropna()  
dfdf=df.nlargest(7, 'imp')  
sns.barplot(x="imp",y="names",hue='names', palette="vlag",data=df,orient="h",ax=ax);

In [ ]:
#Boxplot that shows yield for each item  
a4_dims = (16.7, 8.27)  
  
fig, ax = plt.subplots(figsize=a4_dims)  
sns.boxplot(x="Item",y="hg/ha_yield",hue='hg/ha_yield', palette="vlag",data=df_yield, ax=ax);  